In [ ]:
# Import Libraries
import pandas as pd
import io
import numpy as np
import re

import nltk
nltk.download('wordnet')
nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import wordnet
from itertools import chain
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv('latest_ticket_data.csv')

In [ ]:
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

In [ ]:
df['Description'] = df['Description'].apply(lambda x: get_only_chars(x))

In [ ]:
df['Tokenized']=[nltk.word_tokenize(i) for i in df['Description']]

In [ ]:
df['pos_tagged'] = [nltk.pos_tag(i) for i in df['Tokenized']]

In [ ]:
df['NN_tagged'] = df['pos_tagged'].apply(lambda item:[w for w,t in item if t=='NN'])

In [ ]:
df['NN_Description'] = df.NN_tagged.map(lambda x: ' '.join(x))

In [ ]:
stop_words = stopwords.words('english')
custom_stop_words = ['hi', 'since', 'please', 'best', 'regards', 'thank', 'thanks', 'hello', 'sent', 'great', 'dear', 'help', 'kind']
time_words = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december',
              'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'today' , 'yesterday', 'tomorrow',
              'hour', 'hours', 'time', 'times', 'timelines', 'date', 'day', 'days', 'am', 'pm', 'morning', 'noon', 'afternoon', 'evening',
              'night', 'winter', 'summer', 'rain', 'cold']

def remove_stop_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_custom_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(custom_stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_time_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(time_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

df['NN_Description'] = df["NN_Description"].map(lambda x: remove_stop_words(x))
df['NN_Description'] = df["NN_Description"].map(lambda x: remove_custom_words(x))
df['NN_Description'] = df["NN_Description"].map(lambda x: remove_time_words(x))

In [ ]:
LE = LabelEncoder()
df['label'] = LE.fit_transform(df['Category'])

In [ ]:
def Get_Label_Synonyms(label):
  result=label.split(" ")
  wordcount=len(result)
  synlist = []
  if(wordcount == 1):
    if(wordnet.synsets(label)):
      synonyms = wordnet.synsets(label)
      synlist = list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))
  else:
    for j in range(0,wordcount):
      label_j = []
      if(wordnet.synsets(result[j])):
        synonyms = wordnet.synsets(result[j])
        label_j = list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))
        synlist = synlist + label_j
  synlist = [item.replace("_"," ") for item in synlist]
  return(synlist)

In [ ]:
labelList = df['Category'].unique().tolist()

In [ ]:
def Get_Label_Browns(label):
  result=label.split(" ")
  wordcount=len(result)
  synlist = []
  if(wordcount == 1):
    if(wordnet.synsets(label)):
      synonyms = wordnet.synsets(label)
      synlist = list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))
  else:
    for j in range(0,wordcount):
      label_j = []
      if(wordnet.synsets(result[j])):
        synonyms = wordnet.synsets(result[j])
        label_j = list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))
        synlist = synlist + label_j
  synlist = [item.replace("_"," ") for item in synlist]
  return(synlist)

In [ ]:
def getSynonymKeywords(labelList):
  length=len(labelList)
  topics=[]
  for i in range(0,length):
    label_i = Get_Label_Synonyms(labelList[i])
    t = ' '.join(str(x) for x in label_i)
    u = ' '.join(set(t.split()))
    topics.append(u)
  return(topics)

In [ ]:
def getBrownKeywords(labelList):
  length=len(labelList)
  topics=[]
  for i in range(0,length):
    label_i = Get_Label_Browns(labelList[i])
    t = ' '.join(str(x) for x in label_i)
    u = ' '.join(set(t.split()))
    topics.append(u)
  return(topics)

In [ ]:
def getKeywords(labelList,keywordsType):
  topics=[]
  if(keywordsType=='brown'):
    topics=getBrownKeywords(labelList)
    return(topics)
  if(keywordsType=='synonym'):
    topics=getSynonymKeywords(labelList)
    return(topics)

In [ ]:
topics = getKeywords(labelList,'synonym')
print(topics)

['practical applications coating application programme covering lotion diligence program', 'database', 'web meshwork mesh electronic network net meshing', 'drug exploiter sustenance abuser alimony upkeep maintenance criminal care substance user sustentation sustainment', 'system department security measures measure protection surety certificate']


In [ ]:
# Generate samples that contains K samples of each class
def gen_sample(df, sample_size, num_classes):

    df_1 = df[(df["label"]<num_classes + 1)].reset_index().drop(["index"], axis=1).reset_index().drop(["index"], axis=1)
    train = df_1[df_1["label"] == np.unique(df_1['label'])[0]].sample(sample_size)

    train_index = train.index.tolist()

    for i in range(1,num_classes):
        train_2 = df_1[df_1["label"] == np.unique(df_1['label'])[i]].sample(sample_size)
        train = pd.concat([train, train_2], axis=0)
        train_index.extend(train_2.index.tolist())

    test = df_1[~df_1.index.isin(train_index)]

    return train, test

In [ ]:

brown_ic = wordnet_ic.ic('ic-brown.dat')
dog = wordnet.synset('dog.n.01')
cat = wordnet.synset('cat.n.01')
#dog.res_similarity(cat, brown_ic)
dog.lch_similarity(cat)

[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


2.0281482472922856

In [ ]:
def WordnetShortestPath_labelscore(a,topics):
  lowest_netavg=100
  lowest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.shortest_path_distance(syn2))
            sum=sum+syn1.shortest_path_distance(syn2)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg<lowest_netavg):
      lowest_netavg=netavg
      lowest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return lowest_label,lowest_netavg

In [ ]:
def WordnetLeacock_labelscore(a,topics):
  lowest_netavg=100
  lowest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.shortest_path_distance(syn2))
            sum=sum+syn1.lch_similarity(syn2)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg<lowest_netavg):
      lowest_netavg=netavg
      lowest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return lowest_label,lowest_netavg

In [ ]:
def WordnetRES_labelscore(a,topics):
  highest_netavg=0
  highest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            sum=sum+syn1.res_similarity(syn2, brown_ic)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg>highest_netavg):
      highest_netavg=netavg
      highest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return highest_label,highest_netavg

In [ ]:
def WordnetLIN_labelscore(a,topics):
  highest_netavg=0
  highest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.wup_similarity(syn2))
            sum=sum+syn1.lin_similarity(syn2, brown_ic)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg>highest_netavg):
      highest_netavg=netavg
      highest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return highest_label,highest_netavg

In [ ]:
def WordnetJCN_labelscore(a,topics):
  highest_netavg=0
  highest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.wup_similarity(syn2))
            sum=sum+syn1.jcn_similarity(syn2, brown_ic)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg>highest_netavg):
      highest_netavg=netavg
      highest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return highest_label,highest_netavg

In [ ]:
def WordnetWUP_labelscore(a,topics):
  highest_netavg=0
  highest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.wup_similarity(syn2))
            sum=sum+syn1.wup_similarity(syn2)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg>highest_netavg):
      highest_netavg=netavg
      highest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return highest_label,highest_netavg

In [ ]:
def ZeroShotWordnetModel(text, labelList, keywordstobeGenerated='Yes', keywordsList=[], keywordsType='synonym', posfilterType='NN', measureType='ShortestPath'):
  if not text and not labelList:
    print("Input Text and List of Label/Category names required")
    return()
  else:
    if(keywordstobeGenerated=='Yes'):
      topics = getKeywords(labelList,keywordsType)
    else:
      topics = labelList
    if(posfilterType=='NN'):
      if(measureType=='WuPalmer'):
        return(WordnetWUP_labelscore(text,topics))
      if(measureType=='Resnik'):
        return(WordnetRES_labelscore(text,topics))
      if(measureType=='JCN'):
        return(WordnetJCN_labelscore(text,topics))
      if(measureType=='Lin'):
        return(WordnetLIN_labelscore(text,topics))
      if(measureType=='Leacock'):
        return(WordnetLeacock_labelscore(text,topics))
      if(measureType=='ShortestPath'):
        return(WordnetShortestPath_labelscore(text,topics))


In [ ]:
ZeroShotWordnetModel(df['Description'][0],labelList,measureType='Leacock')

(2, 0.9867316419149126)

In [ ]:
ZeroShotWordnetModel(df['NN_Description'][0],labelList,measureType='Leacock')

(4, 1.0310033595479977)

In [ ]:
ZeroShotWordnetModel(df['Description'][0],labelList,measureType='Lin')

(2, 0.0441047053537972)

In [ ]:
ZeroShotWordnetModel(df['NN_Description'][0],labelList,measureType='Lin')

(2, 0.04571837058225767)

In [ ]:
ZeroShotWordnetModel(df['Description'][0],labelList,measureType='JCN')

(2, 0.04807108388377777)

In [ ]:
ZeroShotWordnetModel(df['NN_Description'][0],labelList,measureType='JCN')

(2, 0.04915213813510252)

In [ ]:
ZeroShotWordnetModel(df['Description'][0],labelList,measureType='Resnik')

(0, 0.47857541687594535)

In [ ]:
ZeroShotWordnetModel(df['NN_Description'][0],labelList,measureType='Resnik')

(0, 0.46481235684567734)

In [ ]:
ZeroShotWordnetModel(df['Description'][0],labelList,measureType='WuPalmer')

(0, 0.22638386011349043)

In [ ]:
ZeroShotWordnetModel(df['NN_Description'][0],labelList,measureType='WuPalmer')

(0, 0.231281816045838)

In [ ]:
ZeroShotWordnetModel(df['Description'][0],labelList)

(2, 10.050925925925927)

In [ ]:
ZeroShotWordnetModel(df['NN_Description'][0],labelList)

(2, 9.451388888888891)

In [ ]:
data, rest = gen_sample(df, 100, 5)

In [ ]:
data[['NN_Leacock_Label', 'NN_Leacock_Value']] = data['NN_Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList,measureType='Leacock')))

In [ ]:
data["NN_Leacock_Label"] = data["NN_Leacock_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_Leacock_Label'])

0.152

In [ ]:
data[['NN_Lin_Label', 'NN_Lin_Value']] = data['NN_Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList,measureType='Lin')))

In [ ]:
data["NN_Lin_Label"] = data["NN_Lin_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_Lin_Label'])

0.226

In [ ]:
data[['NN_JCN_Label', 'NN_JCN_Value']] = data['NN_Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList,measureType='JCN')))

In [ ]:
data["NN_JCN_Label"] = data["NN_JCN_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_JCN_Label'])

0.2

In [ ]:
data[['NN_Resnik_Label', 'NN_Resnik_Value']] = data['NN_Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList,measureType='Resnik')))

In [ ]:
data["NN_Resnik_Label"] = data["NN_Resnik_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_Resnik_Label'])

0.242

In [ ]:
data[['NN_WUP_Label', 'NN_WUP_Value']] = data['NN_Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList,measureType='WuPalmer')))

In [ ]:
data["NN_WUP_Label"] = data["NN_WUP_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_WUP_Label'])

0.224

In [ ]:
data[['NN_WSP_Label', 'NN_WSP_Value']] = data['NN_Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList)))

In [ ]:
data["NN_WSP_Label"] = data["NN_WSP_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_WSP_Label'])

0.196

In [ ]:
data[ ["WSP_Label","WSP_Value"] ] = data["Description"].apply(WordnetShortestPath_labelscore).apply(pd.Series)

In [ ]:
data["WSP_Label"] = data["WSP_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['WSP_Label'])

0.202

In [ ]:
df[['NN_WUP_Label', 'NN_WUP_Value']] = df['Description'].apply(lambda x: pd.Series(ZeroShotWordnetModel(x, labelList)))

In [ ]:
df["NN_WUP_Label"] = df["NN_WUP_Label"].astype(int)

In [ ]:
accuracy_score(df['label'], df['NN_WUP_Label'])

0.206

In [ ]:
data[ ["NN_WUP_Label","NN_WUP_Value"] ] = data["NN_Description"].apply(WordnetWUP_labelscore).apply(pd.Series)

In [ ]:
data["NN_WUP_Label"] = data["NN_WUP_Label"].astype(int)

In [ ]:
data[ ["WUP_Label","WUP_Value"] ] = data["Description"].apply(WordnetWUP_labelscore).apply(pd.Series)

In [ ]:
data["WUP_Label"] = data["WUP_Label"].astype(int)

In [ ]:
accuracy_score(data['label'], data['NN_WUP_Label'])

0.214

In [ ]:
accuracy_score(data['label'], data['WUP_Label'])

0.198

In [ ]:
data

,Description,Category,Tokenized,pos_tagged,NN_tagged,NN_Description,label,NN_WSP_Label,NN_WSP_Value,WSP_Label,WSP_Value,NN_WUP_Label,NN_WUP_Value,WUP_Label,WUP_Value
55,wants principles principles document thank acc...,Application,"[wants, principles, principles, document, than...","[(wants, VBZ), (principles, NNS), (principles,...","[thank, decline]",decline,0,2,7.708333,2,9.201389,2,0.241016,0,0.244646
342,agile hello please add agile kind regards disc...,Application,"[agile, hello, please, add, agile, kind, regar...","[(agile, IN), (hello, JJ), (please, NN), (add,...","[please, kind, lead]",lead,0,2,9.500000,2,10.229167,0,0.245049,0,0.232287
575,card building good morning card working underg...,Application,"[card, building, good, morning, card, working,...","[(card, NN), (building, NN), (good, JJ), (morn...","[card, building, morning, card, please, thank]",card building card,0,2,7.916667,2,8.950000,4,0.317950,0,0.266899
515,same hi regarding complain please pm same,Application,"[same, hi, regarding, complain, please, pm, same]","[(same, JJ), (hi, NN), (regarding, VBG), (comp...","[hi, please, pm]",,0,0,0.000000,2,10.083333,0,0.000000,1,0.220723
221,change hi please change myself thank senior,Application,"[change, hi, please, change, myself, thank, se...","[(change, NN), (hi, NN), (please, NN), (change...","[change, hi, please, change]",change change,0,2,8.625000,2,9.463542,0,0.263633,0,0.236450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612,thursday card hello please help activate card ...,User Maintenance,"[thursday, card, hello, please, help, activate...","[(thursday, JJ), (card, NN), (hello, NN), (ple...","[card, hello, please, floor, resolution, date,...",card floor resolution head,4,2,9.098958,2,9.586310,0,0.247561,0,0.242169
1671,thursday february secure area good morning ple...,User Maintenance,"[thursday, february, secure, area, good, morni...","[(thursday, JJ), (february, JJ), (secure, NN),...","[secure, area, morning, please, area, tower, t...",secure area area tower officer,4,2,9.171875,2,9.607639,1,0.226632,1,0.238750
1638,friday march pm card leaver care firma weekend,User Maintenance,"[friday, march, pm, card, leaver, care, firma,...","[(friday, JJ), (march, NN), (pm, NN), (card, N...","[march, pm, card, care, weekend]",card care weekend,4,2,8.520833,2,9.630952,0,0.305324,0,0.266521
1598,wednesday pm card hello si ca card assignat ca...,User Maintenance,"[wednesday, pm, card, hello, si, ca, card, ass...","[(wednesday, JJ), (pm, VB), (card, NN), (hello...","[card, hello, si, assignat, care, care, si, ca...",card si assignat care care si care va,4,2,8.854167,2,9.013480,0,0.311777,0,0.296457
